In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:
data = pd.read_csv('data/data_fusion_little.csv')
data = data.dropna()
data.head()

,Titre,Description,Auteurs,Image,Lien Google,Editeur,Date publication,infoLink,Genre,Nb scores,Id,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
8,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,ARI272XF8TOL4,Christopher J. Bray,74/81,5.0,955411200,Ecclesiological Milestone,With the publication of Everett Ferguson's boo...
9,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,A36TPZSH8LBT1,haskell,2/3,5.0,1311465600,Early Christian development of the Church,Everett Ferguson approaches the subject of ear...
57,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A30IUAABSEHEPI,Lawrence Duckles,9/9,3.0,1055980800,The Battleship Bismarck reviewed,This book is both a history and a photo album ...
58,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A29SCEAL3JPMYW,Tim Grasshoff,0/0,4.0,1217808000,nice historical pictures,I like this book first one I've seen with so m...
74,Beginner's Yoruba (Hippocrene Beginner's Series),"""Beginner's Yoruba"" is now available with two ...",['Kayode J. Fakinlede'],http://books.google.com/books/content?id=xLe4n...,http://books.google.nl/books?id=xLe4nWzeSw0C&p...,Hippocrene Books,2005,http://books.google.nl/books?id=xLe4nWzeSw0C&d...,['Foreign Language Study'],1.0,0781810698,19.77,A1F0EV2MBF208I,"Olena Y. Rabinowitz ""Book lover""",14/14,4.0,1156291200,Very authentic,This is my first encounter with Yoruba and I h...


In [3]:
from sklearn.model_selection import train_test_split

X_polarite = data.drop(['Image', 'Lien Google', 'infoLink', 'Id', 'revue/score'], axis=1)
Y_polarite = data['revue/score']

X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = train_test_split(X_polarite, Y_polarite, test_size=0.2, random_state=42)

Avec sklearn (sans gpu = temps d'entrainement plus long)

In [4]:
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack
import pandas as pd

# Identifier les types de colonnes
colonnes_categoriques = ['Auteurs', 'Editeur', 'Genre', 'Nom lecteur', 'revue/utilité']
colonnes_numeriques = ['Nb scores', 'Prix', 'revue/heure']
colonnes_textuelles = ['Titre', 'revue/résumé', 'revue/texte']

# Transformer les colonnes catégorielles et numériques
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), colonnes_categoriques),
    ('num', RobustScaler(), colonnes_numeriques),
])

# Appliquer fit_transform sur X_train, mais SEULEMENT transform sur X_test
X_train_cat_num = preprocessor.fit_transform(X_train_polarite)
X_test_cat_num = preprocessor.transform(X_test_polarite)

# Appliquer TF-IDF sur chaque colonne textuelle séparément
text_pipelines = {col: TfidfVectorizer(max_features=500) for col in colonnes_textuelles}

X_train_text = [text_pipelines[col].fit_transform(X_train_polarite[col].fillna("").astype(str)) for col in colonnes_textuelles]
X_test_text = [text_pipelines[col].transform(X_test_polarite[col].fillna("").astype(str)) for col in colonnes_textuelles]

# Concaténer les matrices obtenues pour chaque colonne textuelle
X_train_text_combined = hstack(X_train_text)
X_test_text_combined = hstack(X_test_text)

# Vérification des dimensions après transformation
print(f"Shape X_train_cat_num: {X_train_cat_num.shape}")
print(f"Shape X_test_cat_num: {X_test_cat_num.shape}")
print(f"Shape X_train_text_combined: {X_train_text_combined.shape}")
print(f"Shape X_test_text_combined: {X_test_text_combined.shape}")

# Concaténer toutes les matrices ensemble
if X_train_cat_num.shape[0] == X_train_text_combined.shape[0] and X_test_cat_num.shape[0] == X_test_text_combined.shape[0]:
    X_train_encoded = hstack([X_train_cat_num, X_train_text_combined])
    X_test_encoded = hstack([X_test_cat_num, X_test_text_combined])

    # Vérification finale des dimensions
    print("✅ Shape X_train_encoded:", X_train_encoded.shape)
    print("✅ Shape X_test_encoded:", X_test_encoded.shape)
else:
    print("❌ Problème de dimensions entre les colonnes non-textuelles et textuelles.")

# Enregistre le preprocessor
import pickle
filename = 'preprocesseur/preprocessor.pkl'
pickle.dump(preprocessor, open(filename, 'wb'))


Shape X_train_cat_num: (12776, 21692)
Shape X_test_cat_num: (3195, 21692)
Shape X_train_text_combined: (12776, 1500)
Shape X_test_text_combined: (3195, 1500)
✅ Shape X_train_encoded: (12776, 23192)
✅ Shape X_test_encoded: (3195, 23192)


In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Initialisation du modèle Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, max_depth=6, random_state=42)

# Entraînement
gb_model.fit(X_train_encoded, Y_train_polarite)

# Prédiction
Y_pred_gb = gb_model.predict(X_test_encoded)

# Enregistrement du model
import pickle
filename = 'models/gb_model.pkl'
pickle.dump(gb_model, open(filename, 'wb'))

# Évaluation
mse_gb = mean_squared_error(Y_test_polarite, Y_pred_gb)
print(f"📊 MSE avec Gradient Boosting : {mse_gb:.4f}")

# Enregistrement du modèle
import pickle
filename = 'models/gb_model.pkl'
pickle.dump(gb_model, open(filename, 'wb'))

📊 MSE avec Gradient Boosting : 0.8603


In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

import numpy as np
import pickle

param_grid_lgbm = {
    'n_estimators': [500, 1000],  # Nombre d'arbres
    'learning_rate': [0.01, 0.05],  # Taux d'apprentissage
    'max_depth': [6, 8],  # Profondeur maximale (-1 = illimité)
    'num_leaves': [50, 100],  # Nombre de feuilles dans chaque arbre
    'colsample_bytree': [0.7, 0.5],  # Sélection aléatoire des features par arbre
}

lgbm = LGBMRegressor(device='gpu', random_state=42, boosting_type='gbdt')

grid_search = GridSearchCV(lgbm, param_grid_lgbm, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=8)
grid_search.fit(X_train_encoded, Y_train_polarite)

print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score :", grid_search.best_score_)
print("Meilleur estimateur :", grid_search.best_estimator_)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 133561
[LightGBM] [Info] Number of data points in the train set: 12776, number of used features: 1665
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 28 dense feature groups (0.34 MB) transferred to GPU in 0.002566 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 4.347918
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

TypeError: file must have a 'write' attribute

In [7]:
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

import numpy as np
import pickle

# Initialisation du modèle LightGBM
lgbm_model = LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, num_leaves=100, colsample_bytree=0.5, random_state=42)

# Entraînement
lgbm_model.fit(X_train_encoded, Y_train_polarite)

# Prédiction
Y_pred_lgbm = lgbm_model.predict(X_test_encoded)
print(Y_pred_lgbm)

# Évaluation
mse_lgbm = mean_squared_error(Y_test_polarite, Y_pred_lgbm)
print(f"📊 MSE avec LightGBM : {mse_lgbm:.4f}")

# Enregistrement du modèle
filename = 'models/lgbm_model.pkl'
pickle.dump(lgbm_model, open(filename, 'wb'))


C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 133561
[LightGBM] [Info] Number of data points in the train set: 12776, number of used features: 1665
[LightGBM] [Info] Start training from score 4.347918
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    device='gpu',  # Active le GPU
    random_state=42
)

lgb_model.fit(X_train_encoded, Y_train_polarite)
Y_pred_lgb = lgb_model.predict(X_test_encoded)

mse_lgbm = mean_squared_error(Y_test_polarite, Y_pred_lgb)
print(f"📊 MSE avec LGBM : {mse_lgbm:.4f}")

# Sauvegarde du modèle avec pickle
import pickle
filename = 'models/lgb_model.pkl'
pickle.dump(lgb_model, open(filename, 'wb'))

C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 214086
[LightGBM] [Info] Number of data points in the train set: 139436, number of used features: 4418
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 27 dense feature groups (3.72 MB) transferred to GPU in 0.006433 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 4.219212
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

C:\Users\Cedric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


📊 MSE avec LGBM : 0.7077
